In [1]:
# transformers, torch 등 필요한 패키지 설치
#!pip install pandas torch transformers


In [ ]:
import pandas as pd
import re
import requests
import time
# Google API 설정
api_key = ''


In [3]:
# 주요 유형 한글 매칭 사전
type_translation = {
    "accounting": "회계 사무소",
    "airport": "공항",
    "amusement_park": "놀이공원",
    "aquarium": "수족관",
    "art_gallery": "미술관",
    "atm": "현금인출기",
    "bakery": "제과점",
    "bank": "은행",
    "bar": "술집",
    "beauty_salon": "미용실",
    "bicycle_store": "자전거 상점",
    "book_store": "서점",
    "bowling_alley": "볼링장",
    "bus_station": "버스 정류장",
    "cafe": "카페",
    "campground": "캠핑장",
    "car_dealer": "자동차 딜러",
    "car_rental": "렌터카",
    "car_repair": "자동차 정비소",
    "car_wash": "세차장",
    "casino": "카지노",
    "cemetery": "묘지",
    "church": "교회",
    "city_hall": "시청",
    "clothing_store": "의류점",
    "convenience_store": "편의점",
    "courthouse": "법원",
    "dentist": "치과",
    "department_store": "백화점",
    "doctor": "의사",
    "drugstore": "약국",
    "electrician": "전기공",
    "electronics_store": "전자제품 상점",
    "embassy": "대사관",
    "fire_station": "소방서",
    "florist": "꽃집",
    "funeral_home": "장례식장",
    "furniture_store": "가구점",
    "gas_station": "주유소",
    "gym": "헬스장",
    "hair_care": "헤어 케어",
    "hardware_store": "철물점",
    "hindu_temple": "힌두 사원",
    "home_goods_store": "생활용품점",
    "hospital": "병원",
    "insurance_agency": "보험사",
    "jewelry_store": "보석상",
    "laundry": "세탁소",
    "lawyer": "변호사 사무실",
    "library": "도서관",
    "light_rail_station": "경전철역",
    "liquor_store": "주류 판매점",
    "local_government_office": "지방 정부 기관",
    "locksmith": "자물쇠업체",
    "lodging": "숙박 시설",
    "mosque": "모스크",
    "movie_rental": "영화 대여점",
    "movie_theater": "영화관",
    "moving_company": "이사 업체",
    "museum": "박물관",
    "night_club": "나이트클럽",
    "painter": "페인터",
    "park": "공원",
    "parking": "주차장",
    "pet_store": "애완동물 가게",
    "pharmacy": "약국",
    "physiotherapist": "물리치료사",
    "plumber": "배관공",
    "police": "경찰서",
    "post_office": "우체국",
    "primary_school": "초등학교",
    "real_estate_agency": "부동산",
    "roofing_contractor": "지붕 시공 업체",
    "rv_park": "RV 주차장",
    "school": "학교",
    "secondary_school": "중등학교",
    "shoe_store": "신발 상점",
    "shopping_mall": "쇼핑몰",
    "spa": "스파",
    "stadium": "경기장",
    "storage": "창고",
    "store": "상점",
    "subway_station": "지하철역",
    "supermarket": "슈퍼마켓",
    "synagogue": "유대교 회당",
    "taxi_stand": "택시 승강장",
    "tourist_attraction": "관광 명소",
    "train_station": "기차역",
    "transit_station": "환승역",
    "travel_agency": "여행사",
    "university": "대학",
    "veterinary_care": "수의사",
    "zoo": "동물원"
}

In [4]:
import requests
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# 데이터 파일 불러오기
file_path = 'tn_visit_area_info_방문지정보2nd.csv'
data = pd.read_csv(file_path)

# 결과를 저장할 파일 경로
output_file_path = 'consumecategory.csv'

# VISIT_AREA_TYPE_CD 조건 필터링
valid_types = [4, 8, 10, 11, 12]
data = data[data['VISIT_AREA_TYPE_CD'].isin(valid_types)]

# 기존에 저장된 결과 파일이 있고 비어 있지 않은 경우 불러옴
if os.path.exists(output_file_path) and os.path.getsize(output_file_path) > 0:
    data = pd.read_csv(output_file_path)
else:
    data["CATEGORY"] = None  # CATEGORY 열 추가

# 점심/저녁 식사에 적합한 유형 목록
meal_types = ["restaurant", "meal_delivery", "meal_takeaway"]



def fetch_place_category(area_name, road_address, api_key):
    query = area_name
    response = requests.get(
        "https://maps.googleapis.com/maps/api/place/textsearch/json",
        params={"query": query, "key": api_key, "language": "ko"}
    )
    data = response.json()
    
    # 첫 번째 API 호출 결과 확인
    #print(f"API 응답 (1차): {data}")

    if data["status"] != "OK" or not data.get("results"):
        query = f"{area_name} {road_address}"
        response = requests.get(
            "https://maps.googleapis.com/maps/api/place/textsearch/json",
            params={"query": query, "key": api_key, "language": "ko"}
        )
        data = response.json()

        # 두 번째 API 호출 결과 확인
        #print(f"API 응답 (2차): {data}")
    
    if data["status"] != "OK" or not data.get("results"):
        return "기타", None
    
    place_types = data["results"][0].get("types", [])
    
    for t in place_types:
        if t in meal_types:
            return "음식점", place_types
        elif t in type_translation:
            return type_translation[t], place_types
    
    return "기타", place_types

# 스레드 수 설정: 무료 할당량 기준 ($300)
max_workers = 10  

def process_row(idx, row, api_key):
    # 이미 값이 있는 경우에는 건너뜀
    if pd.notnull(row["CATEGORY"]):
        return idx, row["CATEGORY"]

    category, place_types = fetch_place_category(row["VISIT_AREA_NM"], row["ROAD_NM_ADDR"], api_key)

    # 중간 처리 결과 확인
    #print(f"Row {idx}: VISIT_AREA_NM={row['VISIT_AREA_NM']}, CATEGORY={category}, PLACE_TYPES={place_types}")
    return idx, category

# "CATEGORY" 열이 비어 있는 행만 처리
rows_to_process = data[data["CATEGORY"].isnull()].iterrows()
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_row, idx, row, api_key) for idx, row in rows_to_process]
    
    for future in as_completed(futures):
        idx, category = future.result()
        data.loc[idx, "CATEGORY"] = category
        print(f"{idx + 1}/{len(data)}번째 항목 처리 완료: CATEGORY - {category}")
        
        # 진행 상황을 즉시 저장
        data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"모든 데이터 처리가 완료되었습니다. 결과가 {output_file_path} 파일로 저장되었습니다.")


C:\Users\zx12c\AppData\Local\Temp\ipykernel_30628\2422388155.py:19: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(output_file_path)


310/42390번째 항목 처리 완료: CATEGORY - 음식점
325/42390번째 항목 처리 완료: CATEGORY - 서점
321/42390번째 항목 처리 완료: CATEGORY - 음식점
317/42390번째 항목 처리 완료: CATEGORY - 공원
328/42390번째 항목 처리 완료: CATEGORY - 상점
324/42390번째 항목 처리 완료: CATEGORY - 카페
309/42390번째 항목 처리 완료: CATEGORY - 음식점
320/42390번째 항목 처리 완료: CATEGORY - 카페
316/42390번째 항목 처리 완료: CATEGORY - 음식점
314/42390번째 항목 처리 완료: CATEGORY - 음식점
327/42390번째 항목 처리 완료: CATEGORY - 음식점
323/42390번째 항목 처리 완료: CATEGORY - 기타
315/42390번째 항목 처리 완료: CATEGORY - 카페
312/42390번째 항목 처리 완료: CATEGORY - 관광 명소
296/42390번째 항목 처리 완료: CATEGORY - 기타
319/42390번째 항목 처리 완료: CATEGORY - 음식점
308/42390번째 항목 처리 완료: CATEGORY - 슈퍼마켓
330/42390번째 항목 처리 완료: CATEGORY - 음식점
326/42390번째 항목 처리 완료: CATEGORY - 음식점
311/42390번째 항목 처리 완료: CATEGORY - 음식점
322/42390번째 항목 처리 완료: CATEGORY - 카페
305/42390번째 항목 처리 완료: CATEGORY - 음식점
318/42390번째 항목 처리 완료: CATEGORY - 카페
307/42390번째 항목 처리 완료: CATEGORY - 카페
329/42390번째 항목 처리 완료: CATEGORY - 카페
313/42390번째 항목 처리 완료: CATEGORY - 기타
331/42390번째 항목 처리 완료: CATEGORY - 제과점
332/42390번째